# Going to Mars with Python using poliastro

![poliastro](https://docs.poliastro.space/en/latest/_images/logo_text.png)

## Juan Luis Cano Rodríguez <hello@juanlu.space>

### ...in 5 minutes :)

## *Warning*: This *is* rocket science!

# What is Astrodynamics?

> A branch of Mechanics (itself a branch of Physics) which studies practical problems concerning the motion of rockets and spacecraft throght space

![Rosetta's mission](http://www.esa.int/var/esa/storage/images/esa_multimedia/images/2014/10/rosetta_s_journey_and_timeline/14978088-1-eng-GB/Rosetta_s_journey_and_timeline_node_full_image_2.jpg)

# What is poliastro?

> A pure Python library for Astrodynamics

# Let's go to Mars!

Perseverance trajectory:

![Perseverance trajectory](perseverance.png)

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np

import astropy.units as u
from astropy import time

In [ ]:
# https://github.com/plotly/plotly.py/issues/1664#issuecomment-511773518
import plotly.graph_objects as go
import plotly.io as pio

# Set default renderer
pio.renderers.default = "plotly_mimetype+notebook_connected+jupyterlab"

# Set default template
pio.templates["slides"] = go.layout.Template(layout=dict(width=700, height=550))
pio.templates.default = "plotly+slides"

## Preliminaries: How to define an orbit

In [ ]:
from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

r = [-6045, -3490, 2500] * u.km
v = [-3.457, 6.618, 2.533] * u.km / u.s

ss = Orbit.from_vectors(Earth, r, v)
ss

In [ ]:
ss.plot(interactive=True, label="Initial orbit")

## First: locate the planets

In [ ]:
from astropy.coordinates import solar_system_ephemeris

solar_system_ephemeris.set("jpl")

### Earth

In [ ]:
date_launch = time.Time("2020-07-30 11:50", scale="utc")

earth_launch = Orbit.from_body_ephem(Earth, date_launch)
earth_launch

In [ ]:
earth_launch.r

In [ ]:
earth_launch.v

### Mars

In [ ]:
date_arrival = time.Time("2021-02-18 20:55", scale="utc")

tof = date_arrival - date_launch

mars_arrival = Orbit.from_body_ephem(Mars, date_arrival)
mars_arrival

## Second: Compute the trajectory

In [ ]:
from poliastro.maneuver import Maneuver

man_lambert = Maneuver.lambert(earth_launch, mars_arrival)

# Get the transfer orbit
ss_trans, _ = earth_launch.apply_maneuver(man_lambert, intermediate=True)
ss_trans

In [ ]:
from poliastro.twobody.propagation import propagate

tofs = time.TimeDelta(np.linspace(0 * u.h, tof.to(u.h), num=100))

ss_trans_traj = propagate(
    ss_trans,
    tofs,
)
ss_trans_traj

### ...and it is pure Python!

Trick: **numba**

![Fortran and Python](benchmarks.png)

From my paper Cano Rodríguez, Juan Luis, Helge Eichhorn, and Frazer McLean: poliastro: An Astrodynamics library written in Python with Fortran performance.

## Third: Go to Mars!

In [ ]:
from poliastro.plotting import OrbitPlotter3D

plotter = OrbitPlotter3D()

In [ ]:
plotter.plot(earth_launch, label="Earth at launch position", color="navy")
plotter.plot(mars_arrival, label="Mars at arrival position", color="red")
plotter.plot_trajectory(
    ss_trans.sample(max_anomaly=145 * u.deg), color="black", label="Transfer orbit"
)
plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

## Fourth: Make it interactive!!!1!

In [ ]:
from ipywidgets import interact, interactive, FloatSlider, VBox
from plotly.graph_objects import FigureWidget

In [ ]:
def go_to_mars(tof_h):
    date_arrival = date_launch + tof_h * u.h
    mars_arrival = Orbit.from_body_ephem(Mars, date_arrival)

    man_lambert = Maneuver.lambert(earth_launch, mars_arrival)

    # Get the transfer orbit
    ss_trans, _ = earth_launch.apply_maneuver(man_lambert, intermediate=True)

    tofs = time.TimeDelta(np.linspace(0 * u.h, tof_h * u.h, num=100))
    ss_trans_traj = propagate(
        ss_trans,
        tofs,
    )

    with fig.batch_update():
        # FIXME: Hack to animate the figure seamlessly
        # that however adds more and more invisible traces to the figure
        for trace in fig.data:
            if "Earth at launch position" not in trace["name"]:
                trace.visible = False

        plotter.plot(mars_arrival, label="Mars at arrival position", color="red")
        plotter.plot_trajectory(ss_trans_traj, color="black", label="Transfer orbit")

In [ ]:
fig = FigureWidget()
fig.layout.title = "MSL Mission: from Earth to Mars"

# Plot Earth and set the view in advance
plotter = OrbitPlotter3D(figure=fig)
plotter.plot(earth_launch, label="Earth at launch position", color="navy")
plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

slider = interactive(
    go_to_mars,
    tof_h=FloatSlider(
        min=5_000, value=6_000, max=12_000, step=250, continuous_update=False
    ),
)

In [ ]:
VBox([slider, fig])

## Thank you very much!

### https://docs.poliastro.space
### <hello@juanlu.space>

![Rocket](http://www.esa.int/var/esa/storage/images/esa_multimedia/images/2012/02/liftoff_of_vega_vv014/9659852-3-eng-GB/Liftoff_of_Vega_VV01_large.jpg)